# Install

In [ ]:
!pip install scikit-learn==1.4.2 transformers==4.43.0 datasets==2.20.0 peft==0.10.0 accelerate==0.32.1 bitsandbytes==0.43.1 trl==0.9.6

# 7.4 QLoRA 미세조정

## 모델

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
)

model_name = "nlpai-lab/ko-gemma-2b-v1"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="float16",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

gen_cfg = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 데이터셋

In [2]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")
dataset["train"][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [3]:
import torch

doc = dataset["train"]["question"][0]
messages = [{"role": "user", "content": f"다음 질문에 대해 대답해 주세요.\n질문 : {doc}"}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
)

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))

<bos><start_of_turn>user
다음 질문에 대해 대답해 주세요.
질문 : 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?<end_of_turn>
<start_of_turn>model
북태평양 기단과 오호츠크 해양 기단이 만나면서 전 세계에 바로 가는 지점은 아무 곳도 없습니다. 북태평양 기단은 북극을 향해 이동하고 있으며, 오호츠크 해양 기단은 아시아 서부 지역을 따라 이동하고 있습니다. 이러한 기단들이 국내에 머무르는 기간은 주로 이동의 방향에 따라 달라집니다. 북태평양 기단은 북극으로 향하고 오호츠크 해양 기단은 아시아 서부로 향합니다. 북태평양 기단이 북극으로 향한 경우, 미국 북서부 지역에 머무르며, 오호츠크 해양 기단이 아시아 서부 지역으로 향한 경우, 남동쪽 미국 중부 지역에 머무르게 됩니다. 이러한 방향에 따라 지속된 기간은 다양할 수 있습니다.<eos>


## QLoRA 미세조정 준비

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=bnb_config,
)
model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
def preprocess(example):
    result = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": example["question"]},
            {"role": "model", "content": example["answers"]["text"][0]},
        ],
        tokenize=True,
        return_dict=True,
    )
    return result

# 전처리 할때만 eos 토큰 추가
added_template = "{{ eos_token }}"
tokenizer.chat_template += added_template

dataset = dataset.map(
    preprocess,
    num_proc=2,
    remove_columns=dataset["train"].column_names,
)

# 전처리 후 다시 제거
tokenizer.chat_template = tokenizer.chat_template[:-len(added_template)]

Map (num_proc=2):   0%|          | 0/17554 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/5841 [00:00<?, ? examples/s]

## QLoRA 학습

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling


args = TrainingArguments(
    output_dir="/content/ckpt",
    max_steps=3000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    warmup_ratio=0.03,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to="tensorboard",
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    data_collator=collator,
)
trainer.train()

## 모델 저장

In [ ]:
model.save_pretrained("/content/ko-gemma-2b-sum-v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
!ls -alh /content/ko-gemma-2b-sum-v1

total 38M
-rw------- 1 root root  726 Jul 14 15:36 adapter_config.json
-rw------- 1 root root  38M Jul 14 15:36 adapter_model.safetensors
-rw------- 1 root root 5.0K Jul 14 15:36 README.md


## 추론

In [ ]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    GenerationConfig, 
    BitsAndBytesConfig
)
from peft import PeftModel
from datasets import load_dataset

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)
model = PeftModel.from_pretrained(
    model,
    "/content/ko-gemma-2b-sum-v1",
    device_map="auto",
    torch_dtype="float16",
)
model = model.merge_and_unload()

# datasets
dataset = load_dataset("klue", "mrc")
dataset["train"][0]

doc = dataset["train"]["question"][0]
messages = [{"role": "user", "content": doc}]

# generate
gen_cfg = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
)
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
)
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<bos><start_of_turn>user
다음 질문에 대해 대답해 주세요.
질문 : 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?<end_of_turn>
<start_of_turn>model
13일<end_of_turn><eos>
